##Wstęp
Poniższy raport podejmuje problem optymalnego rozmieszczenia satelitów na orbicie ziemskiej. W szczególności, przedmiotem zainteresowania są satelity poruszające się po orbitach heliosynchronicznych – czyli pojawiające się nad danym punktem Ziemi zawsze o tej samej godzinie słonecznego czasu lokalnego – i jednocześnie będące satelitami obserwacyjnymi, a zatem satelitami, które dzięki wyposażeniu w kamery, są zdolne do tworzenia zdjęć Ziemi w różnych pasmach optycznych z wysoką rozdzielczością.

Nasze główne pytania badawcze brzmiały następująco: Ile trzeba być mieć takich satelitów i jak rozmieszczonych, żeby mieć obrazek dowolnego fragmentu Ziemi co 2 albo 3 godziny? Jaka jest zależność między częstotliwością rewizyty, a liczbą satelitów? Jak zaimplementować rozwiązanie tego problemu, by niewielkim kosztem uzyskać rozwiązanie również dla Księżyca lub innych planet?

W celu odpowiedzi na powyższe pytania, stworzyliśmy pakiet w Pythonie, który m. in. umożliwia tworzenie obiektów klasy Planeta, Orbita i Satelita, jak również umożliwia niezbędne obliczenia fizyczne w celu symulowania ich ruchu i trasy, który dany satelita pokona. Na problem optymalizacyjny spojrzeliśmy dwojako: z jednej strony, sprawdziliśmy rozwiązanie wyliczone ręcznie, które, mimo swojej prostoty, daje wstępne oszacowanie liczby i rozmieszczenia potrzebnych satelitów. Następnie dodaliśmy do naszego pakietu rozwiązanie wykorzystujące algorytm genetyczny, by sprawdzić, czy dzięki komputerowym technikom optymalizacyjnym uda się zmniejszyć liczbę satelitów, a tym samym obniżyć koszty, które poniesie firma.


## Podstawowe działanie satelity

Większość satelitów obserwacyjnych porusza się na niskich orbitach, a wysokości ok. 500-600 km od powierzchni Ziemi. Zazwyczaj wykorzystuje się orbity polarne heliosynchroniczne, co ułatwia uzyskanie pełnego pokrycia Ziemi.

Satelity obserwacyjne wypełniają wiele ważnych zadań, a zbierane przez nie dane są wykorzystywane m.in. w badaniach dotyczących zarówno zmian klimatu, pogody, mapowania terenu, jak i  monitorowania stanu środowiska naturalnego, co obejmuje m.in. badanie mórz i oceanów, stanu wegetacji ziemskiej czy lodowców.

W 2021 r. na orbitach ziemskich znajdowało się ok. 950 satelitów obserwacyjnych.

Przydatne pojęcia:
1. **Rozdzielczość przestrzenna**  -- najmniejszy obszar na powierzchni Ziemi, który może być rozróżniony przez sensor satelity jako oddzielna jednostka. Jest zwykle wyrażana metrach i jest związana z rozmiarem piksela na obrazie satelitarnym.

2. **Pas satelity (swath)** -- obszar na powierzchni Ziemi, który jest obserwowany przez satelitę podczas jednego przelotu


## Fizyka satelity
Satelita to obiekt krążacy wokół innego ciała pod wpływem siły grawitacji pełniącej rolę siły dośrodkowej. Aby satelita mógł krążyć na stałej wysokości nad Ziemią, możemy obliczyć potrzebną prędkość, korzystając ze wzoru
$v = \sqrt{\frac{G \times M}{R + h}}$, gdzie $h$ to wykość satelity nad powierzchnią Ziemii, $G$, to stała grawitacyjna, $M$- masa Ziemii oraz $R$ to promień naszej planety.
Każdy satelita krąży wokół obiektu po pewnej orbicie. Jednym z jej przykładów jest orbita heliosynchroniczna, co oznacza, że jej płaszczyzna jest pod tym samym kątem względem słońca cały rok. Ze względu na jej charakterystykę satelity krążące nad Ziemią  po orbitach heliosynchronicznych latają na wysokościach 600-800km [4] oraz okrążają planetę co około 90-100 minut.
W naszym rozwiązaniu będziemy opisywali orbitę kilkoma parametrami:

  * Wielka półoś elipsy, po której się porusza
  * Ekscentryczność- miara “okrągłości” orbity. Idealnie okrągła orbita ma ekscentryczność równą zero, zaś wyższe wartości wskazują na bardziej "spłaszczone" orbity
  * Inklinacja-  kąt między płaszczyzną orbity a płaszczyzną odniesienia [5]
  * RAAN -  kąt mierzony między kierunkiem równonocy wiosennej a węzłem wstępującym, czyli punktem, w którym satelita przekracza równik, poruszając się z południa na północ.
  * Perygeum- kąt między węzłem wstępującym a perygeum, mierzony w płaszczyźnie orbity w kierunku ruchu satelity.
  * Prawdziwa anomalia- kąt między kierunkiem periapsis a aktualną pozycją ciała, widziany z głównego ogniska elipsy (punktu, wokół którego obiekt krąży).

<div>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/c2/Orbit1_pl.svg/1200px-Orbit1_pl.svg.png" width="500"/>
</div>
Każdy satelita w projekcie ma robić zdjęcia wysokiej rozdzielczości. Porównując najpopularniejsze ze spełniających wymagania Zleceniodawcy [1], [2] dowiadujemy się, że robią one zdjęcia obszarów z przedziału 15-20 km szerokości, natomiast mogą się poruszać po orbicie o rzędu kilkuset kilometrów w ciągu kilkunastu sekund.
Na ruch satelity wpływ mają również między innymi efekt precesji [6] czy współczynnik spłaszczenia (dla Ziemii wynosi on około $1.083 \times 10^{-3}$) [7] co również zostało uwzględnione w naszych symulacjach.

Oprócz satelitów, istotnymi komponentami projektu były także odbiorniki. Są one w stanie odbierać sygnał od satelitów wówczas gdy nadawca znajduje się powyżej płaszczyzny stycznej do planety w miejscu odbiornika, dzięki czemu możemy w łatwy sposób za pomocą prostych rachunków oraz metody Haversine obliczyć czy dany satelita jest widoczny przez dany odbiornik czy też nie.


# Jak propagujemy orbitę?

- $\mu$: Standardowy parametr grawitacyjny planety $(m^3/s^2)$.
- $a$: Półoś wielka orbity (semi-major axis of the orbit) $(m)$.
- $e$: Mimośród orbity (eccentricity).
- $\theta$: Prawdziwa anomalia (true anomaly) $(rad)$, kąt od kierunku perycentrum do aktualnej pozycji satelity.
- $\Omega$: Prawoskrętność wznoszenia węzła wstępującego (Right Ascension of the Ascending Node, RAAN) $(rad)$.
- $i$: Inklinacja orbity (Inclination of the orbit) $(rad)$.
- $\omega$: Argument periapsis (Argument of periapsis) $(rad)$.

Proces obejmuje dwa główne kroki:

**Krok 1: Oblicz Pozycję i Prędkość w Współrzędnych Peryfokalnych**

1. Pół-latus rectum (Semi-latus rectum):
$$ p = a(1 - e^2) $$

2. Promień (Radius, odległość od punktu ogniskowego do satelity):
$$ r = \frac{p}{1 + e\cos(\theta)} $$

3. Wektor pozycji w układzie PQW:
$$ \vec{r}_{pqw} =
\begin{bmatrix}
r\cos(\theta) \\
r\sin(\theta) \\
0
\end{bmatrix}
$$

4. Wektor prędkości w układzie PQW:
$$ \vec{v}_{pqw} =
\begin{bmatrix}
-\sqrt{\frac{\mu}{p}}\sin(\theta) \\
\sqrt{\frac{\mu}{p}}(e + \cos(\theta)) \\
0
\end{bmatrix}
$$

**Krok 2: Obrót do w układzie ECI (biorąc ziemię za układ odniesienia) za pomocą trzech macierzy rotacji**

1. Obrót wokół osi Z o $-\Omega$ (RAAN):
$$ R_3(-\Omega) =
\begin{bmatrix}
\cos(-\Omega) & -\sin(-\Omega) & 0 \\
\sin(-\Omega) & \cos(-\Omega) & 0 \\
0 & 0 & 1
\end{bmatrix}
$$

2. Obrót wokół osi X o $-i$ (inclination):
$$ R_1(-i) =
\begin{bmatrix}
1 & 0 & 0 \\
0 & \cos(-i) & -\sin(-i) \\
0 & \sin(-i) & \cos(-i)
\end{bmatrix}
$$

3. Obrót wokół osi Z o $-\omega$ (argument of perigee):
$$ R_3(-\omega) =
\begin{bmatrix}
\cos(-\omega) & -\sin(-\omega) & 0 \\
\sin(-\omega) & \cos(-\omega) & 0 \\
0 & 0 & 1
\end{bmatrix}
$$

**Ostateczna Pozycja i Prędkość w Ramie ECI (Final Position and Velocity in ECI Frame):**
$$ \vec{r}_{eci} = R_3(-\Omega)R_1(-i)R_3(-\omega)\vec{r}_{pqw} $$
$$ \vec{v}_{eci} = R_3(-\Omega)R_1(-i)R_3(-\omega)\vec{v}_{pqw} $$



Funkcja `propagate_orbit` symuluje orbitę satelity w czasie, uwzględniając zarówno główną atrakcję grawitacyjną planety, jak i zakłócenie spowodowane spłaszczeniem planety (zakłócenie J2). Proces ten obejmuje rozwiązanie układu równań różniczkowych opisujących ruch satelity pod wpływem tych sił.

### Problem dwóch ciał z zakłóceniem J2

Problem dwóch ciał zwykle opisuje oddziaływanie grawitacyjne między dwoma ciałami, zakładając, że są one masami punktowymi, co prowadzi do orbit eliptycznych. Jednak rzeczywiste ciała niebieskie nie są masami punktowymi ani doskonałymi sferami, co prowadzi do dodatkowych efektów perturbacyjnych na orbicie satelity. Jedno z istotnych zakłóceń pochodzi od terminu J2, który uwzględnia wypukłość równikową planety spowodowaną jej rotacją.

#### Czym jest zakłócenie J2?

Zakłócenie J2 uwzględnia niesferyczny kształt planety. Potencjał grawitacyjny ciała z wypukłością równikową różni się od tego dla doskonałej sfery, co prowadzi do zmian w orbicie satelity w czasie, szczególnie wpływając na jej inklinację i prawoskrętność wznoszenia węzła wstępującego (RAAN).

### Sformułowanie matematyczne

Mając dane:
- $ \mu $: Standardowy parametr grawitacyjny planety $(m^3/s^2)$.
- $ J2 $: Bezmiarowy współczynnik reprezentujący wypukłość równikową planety.
- $ r $: Wektor pozycji satelity.
- $ v $: Wektor prędkości satelity.
- $ R $: Średni promień planety.

#### Równania różniczkowe dla propagacji orbity

Ruch satelity jest regulowany przez drugie prawo Newtona, z przyspieszeniem danym przez gradient potencjału grawitacyjnego. Uwzględniając zakłócenie J2, przyspieszenie $ a $ jest:
$$ \vec{a} = \vec{a}_{gravity} + \vec{a}_{J2} $$

Gdzie:
- $ \vec{a}_{gravity} = -\frac{\mu}{r^3} \vec{r} $ jest głównym przyspieszeniem grawitacyjnym.
- $ \vec{a}_{J2} $ uwzględnia zakłócenie spowodowane przez termin J2.

Wyraźnie, składowe przyspieszenia J2 w kartezjańskich współrzędnych są:
$$ \vec{a}_{J2} = \frac{1.5 J2 \mu R^2}{r^4}
\left( \begin{array}{c}
(1 - 5\frac{z^2}{r^2})x \\
(1 - 5\frac{z^2}{r^2})y \\
(3 - 5\frac{z^2}{r^2})z
\end{array} \right)
$$

Ta formuła dostosowuje przyspieszenie satelity ze względu na wypukłość równikową, wpływając na ewolucję orbity.

#### Numeryczne rozwiązanie problemu

Aby propagować orbitę, numerycznie całkujemy równania ruchu za pomocą `solve_ivp`. Początkowy wektor stanu ($ \vec{y}_0 = [\vec{r}, \vec{v}] $) jest określany na podstawie aktualnych elementów orbitalnych satelity. Równania różniczkowe zdefiniowane przez $ \vec{a} $ są całkowane na żądanym przedziale czasu, dając trajektorię satelity.

### Obliczanie ścieżki naziemnej

Metoda `calculate_ground_track` przekształca trajektorię satelity z ramy ECI na współrzędne geograficzne (szerokość i długość geograficzna). Obliczenie to uwzględnia rotację Ziemi, pozwalając na określenie, gdzie satelita będzie umiejscowiony względem powierzchni planety w czasie.

Transformacja obejmuje:
1. Obliczanie długości geograficznej przez dostosowanie długości geograficznej ECI satelity przez rotację Ziemi w czasie.
2. Bezpośrednie obliczanie szerokości geograficznej z koordynatu z ECI satelity.
3. Dostosowanie długości geograficznej, aby upewnić się, że pozostaje ona w zakresie $[-180^\circ, 180^\circ]$.

Ta metoda dostarcza ścieżkę naziemną satelity, która jest kluczowa dla planowania misji, w tym okien obserwacyjnych i komunikacyjnych.

Podsumowując, `propagate_orbit` i `calculate_ground_track` razem oferują kompleksową symulację orbit satelitów, uwzględniając zarówno główne siły grawitacyjne, jak i zakłócenia spowodowane przez kształt planety, oraz mapując ścieżkę satelity nad powierzchnią planety.


## Wstępne rozwiązanie
Wstępne rozwiązanie do którego będziemy się porównywali w przyszłości:
Na początku spóbujmy oszacować ile satelitów potrzebowalibyśmy. Skoro satelita Plesiades będzie w stanie zbierać obrazy w dowolnym miejscu w pasie naziemnym o szerokości 800 km, pokonując 200 km w 11 s lub 800 km w 25 s [2], wliczając w to czas stabilizacji, to skoro okrąża Ziemię około 12 razy na dobę (raz na dwie godziny), to znaczy, że jego "widok" na 15min, to $800\text{km} \times (40 000 / (2 \times 4)) = 800 \times 5000 = 4000000 \text{km}^2$
Ponieważ powierzchnia ziemii to 510 100 000 km$^2$, czyli potrzeba około 510 100 000 / 4000000 = 128 satelitów.
Aby zobrazować, że jest to wystarczająca liczba satelitów stworzyliśmy skrypt pozwalający zwizualizować trasy każdego z satelitów na wybranych orbitach przez 15 minut. Biorąc pod uwagę, że każdy z tych satelitów okrąża Ziemię w ciągu około 100 minut i przelatuje w pobliżu bieguna, zawsze będziemy mieli możliwość przesyłania plików do danego odbiornika.


In [2]:

#należy wcześniej zainstalować bibliotekę poliastro
from astropy import units as u
import numpy as np
import pandas as pd
from poliastro.earth import EarthSatellite
from poliastro.earth.plotting import GroundtrackPlotter
from poliastro.examples import iss
from poliastro.util import time_range
from poliastro.bodies import Earth
from poliastro.twobody import Orbit
import math
import random
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'poliastro'

## Jak sprawdzamy, czy satelity są widzialne przez stacje?

Korzystając z metody haversine, która wylicza odległość kątową między dwoma punktami na sferze.

In [ ]:
#funkcja zwracająca odległość kątową między dwoma punktami
def haversine(r_long, r_lat, s_long, s_lat, planet_radius): #(długość geograficzna 1. punktu, szerokość geo. 1. pkt, długość geo. 2. pkt, szer. geo. 2. pkt, promien sfery)
    return 2 * planet_radius * np.arcsin(np.sqrt( (1 - np.cos(s_lat - r_lat) + np.cos(s_lat) * np.cos(r_lat) * (1 - np.cos(s_long - r_long))) / 2   ))

#kąt między satelitą, a płaszczyzną styczną do powierzchni sfery w miejscu odbiornika, który mówi czy satelita jest widziany przez odbiornik
def seeable_angle(planet_radius, s_alt):
    return np.arccos(planet_radius / (planet_radius + s_alt))

#funkcja obliczające czy dany satelita jest widoczny przed dany odbiornik
def is_vis2(receiver_long, receiver_lat, satellite_lat, satellite_long, satellite_alt, planet_radius):
    return np.fabs(haversine(receiver_long, receiver_lat, satellite_long, satellite_lat, planet_radius)) < np.fabs(seeable_angle(planet_radius, satellite_alt))


In [ ]:
#funkcja wyliczająca obszar, z którego odbiornik potrafi odbierać sygnał od satelitów oddalonych od powierzchni planety o satellite_alt km.
def find_visible_circle(receiver_long, receiver_lat, satellite_alt, planet_radius):
    #nie wpisywać biegunów, bo tam są błędy zmiennoprzecinkowe chyba
    if receiver_lat == 90:
        receiver_lat = 89.99
    elif receiver_lat == -90:
        receiver_lat = -89.99
    distance_rad = seeable_angle(planet_radius, satellite_alt)
    long_rad = math.radians(receiver_long)
    lat_rad = math.radians(receiver_lat)

    random_points = []
    m = 100
    for i in range(m):

        bearing = ((2 * math.pi * i )/ m) - math.pi

        new_lat_rad = math.asin(math.sin(lat_rad)*math.cos(distance_rad) +
                                math.cos(lat_rad)*math.sin(distance_rad)*math.cos(bearing))
        new_long_rad = long_rad + math.atan2(math.sin(bearing)*math.sin(distance_rad)*math.cos(lat_rad),
                                             math.cos(distance_rad)-math.sin(lat_rad)*math.sin(new_lat_rad))

        new_lat = math.degrees(new_lat_rad)
        new_long = math.degrees(new_long_rad) - receiver_long

        random_points.append((new_long, new_lat))

    return random_points

## Wizualizacja
Do pełnego zrozumienia działania algorytmu napisaliśmy skrypt pozwalający zwizulaizować trasy danego satelity lub całej konstelacji.
W tym celu korzystaliśmy z biblioteki poliastro.

Mogliśmy dzięki niej sprawdzić działanie funkcji calculate_orbit_parameters, która miała podać nam parametry orbity chcąc, aby satelita przelatywał zawsze o tej samej godzinie słonecznej nad danym punktem.

In [ ]:
def calculate_orbit_parameters(latitude_deg, longitude_deg, altitude_m, local_time_h):
    # Constants
    mu = 398600.4418  # Earth's gravitational parameter, km^3/s^2
    R_earth = 6371  # Earth's radius, km
    J2 = 1.08263e-3  # Earth's J2 value
    inclination = np.radians(98.6)  # Typical SSO inclination in radians

    # Convert altitude from meters to kilometers
    altitude_km = altitude_m / 1000

    # Calculate semi-major axis
    semi_major_axis = R_earth + altitude_km

    # Orbital period (in seconds)
    T = 2 * np.pi * np.sqrt(semi_major_axis ** 3 / mu)

    # Approximate local solar time to RAAN conversion
    # This is a simplified model and doesn't account for the exact dynamics
    # Assuming the satellite crosses the equator at the local noon (maximum simplification)
    GMT_offset = longitude_deg / 15  # Convert longitude to GMT offset
    RAAN_offset_hours = local_time_h - 12  # Difference from local noon
    daily_precession_deg = 360 / 365.25  # Daily precession needed for SSO
    RAAN = (GMT_offset + RAAN_offset_hours) * 15 - daily_precession_deg  # Simplified RAAN calculation

    # Ensure RAAN is within 0 to 360 degrees
    RAAN = RAAN % 360

    return {
        "semi_major_axis_km": semi_major_axis,
        "inclination_deg": np.degrees(inclination),
        "RAAN_deg": RAAN,
        "period_s": T
    }


# Example usage: Calculate parameters for a satellite to pass over Warsaw at 10:00 local time
latitude_deg = 52.2297
longitude_deg = 21.0122
altitude_m = 700000  # 700 km altitude
local_time_h = 10.0  # 10:00 AM

orbit_params = calculate_orbit_parameters(latitude_deg, longitude_deg, altitude_m, local_time_h)
print(orbit_params)

{'semi_major_axis_km': 7071.0, 'inclination_deg': 98.6, 'RAAN_deg': 350.02657371663247, 'period_s': 5917.417835241439}


In [ ]:
a = 7071 * u.km
inc = 98.6 * u.deg
raan = 350.02657371663247 * u.deg
orbits = []
new_satelites = []
times = []
around = 23.934 #ile godzin trwa obrót ziemii wokół własnej osi
sun_hour = (1.0 / 24.0) * around
n = 30 #liczba dni
pleiades_orbit = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu, epoch)
for i in range(n):
    orbits.append(Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu, epoch))
    new_satelites.append(EarthSatellite(orbits[i], None))
    times.append(time_range(
        pleiades_orbit.epoch + (i * around + sun_hour * 11) * u.h, periods=150, end=pleiades_orbit.epoch + (i * around + sun_hour * 11.25 ) * u.h ))

# Build spacecraft instance
pleiades_spacecraft = EarthSatellite(pleiades_orbit, None)

# Generate an instance of the plotter, add title and show latlon grid
gp = GroundtrackPlotter()
gp.update_layout(title="Warszawska orbita")

from colour import Color
red = Color("red")
colors = list(red.range_to(Color("green"),n))

# Plot previously defined EarthSatellite object
for i in range(n):
    gp.plot(
        new_satelites[i],
        times[i],
        label=str(i + 1) + "-ty dzień",
        color=str(colors[i]),
        line_style = {"width": 3 , "color": str(colors[i])},
        marker={
            "size": 0,
            "symbol": "triangle-right",
            "line": {"width": 0, "color": "black"},
        },
    )
# For building geo traces
import plotly.graph_objects as go

# Let us add a new trace in original figure
WWA = [52.2297, 21.0122] * u.deg
gp.add_trace(
    go.Scattergeo(
        lat=WWA[0],
        lon=WWA[-1],
        name="WARSZAWA",
        marker={"color": "blue"},
        )
)

gp.fig.show()

In [ ]:
gp.update_geos(projection_type="orthographic")
gp.fig.show()

Dzięki temu mogliśmy też sprawdzić czy wstępne rozwiązanie rzeczywiście pokryłoby całą planetę dobierając orbity ręcznie. Każdy odcinek to trasa jednego satelity przez 15 minut,

In [ ]:
# Create the orbit
orbits = []
new_satelites = []
times = []
n = 128
pleiades_orbit = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu, epoch)
for i in range(n):
    orbits.append(Orbit.from_classical(Earth, a, ecc, inc, raan, (argp * i) / float(n), nu, epoch))
    new_satelites.append(EarthSatellite(orbits[i], None))
    times.append(time_range(
        pleiades_orbit.epoch + ( i * 0.25 )  * u.h, periods=150, end=pleiades_orbit.epoch + ( (i + 1) * 0.25) * u.h ))
pleiades_orbit = Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu, epoch)

# Build spacecraft instance
pleiades_spacecraft = EarthSatellite(pleiades_orbit, None)

gp = GroundtrackPlotter()
gp.update_layout(title="Wstępne rozwiązanie")

from colour import Color
red = Color("red")
colors = list(red.range_to(Color("green"),n))

# Plot previously defined EarthSatellite object
for i in range(n):
    gp.plot(
        new_satelites[i],
        times[i],
        label="Wstępne" + str(i),
        color=str(colors[i]),
        line_style = {"width": 3 , "color": str(colors[i])},
        marker={
            "size": 0,
            "symbol": "triangle-right",
            "line": {"width": 0, "color": "black"},
        },
    )
# For building geo traces
import plotly.graph_objects as go

# Position in [LAT LON]
STATIONS = []
num_st = 4
for j in range(num_st):
    STATION = [((360 * j) / num_st) - 180, ((360 * j) / num_st) - 180] * u.deg
    STATIONS.append(STATION)

for j in range(num_st):
    gp.add_trace(
        go.Scattergeo(
            lat=STATIONS[j][0],
            lon=STATIONS[j][-1],
            name="Odbiornik",
            marker={"color": "black"},
            )
    )
gp.fig.show()

In [ ]:
gp.update_geos(projection_type="orthographic")
gp.fig.show()

Powyższe obrazki miały na celu wizualizację zrozumienia w jaki sposób wyglądają orbity typowych satelitów oraz sprawdzenia czy nasze funkcje rzeczywiście działają w porządany sposób.

## Algorytm genetyczny

Pełne kody można znaleźć [w naszym repozytorium na github](https://github.com/atanazy-tot/uzytki-proj2/tree/alg_gen). Do uruchomienia symulacji wystarczy uruchomić plik `main.py` z wybranymi przez siebie parametrami algorytmu. Program korzysta z następujących bibliotek -- jeżeli aktualnie nie są zainstalowane, konieczne będzie ich pobranie:

*  `math`
*  `random`
*  `numpy`
*  `scipy`
*  `shapely`

Przykładowe użycie:

In [ ]:
# NOT RUN -- just an example, will not work
gen_alg = GeneticAlgorithm(planet=earth,
                          constellation_size=128,
                          popSize=20,
                          numParents=8,
                          pm=0.3,
                          time_span=900,
                          swath_width=800,
                          numIterations=15,
)

gen_alg.geneticAlgorithm(cross_1_point=False, file_name = "sat_params.txt")

Powyższa konfiguracja uruchodmi algorytm genetyczny poszukujący optymalnej konstelacji złożonej ze 128 satelit, rozważając populacje złożone z 20 osobników (konstelacji), krzyżując w każdej iteracji 8 konstelacji, mutując konstelacje z prawdopodobieńśtwem 0.3. Jednocześnie, interesuje nas w tym przypadku pokrycie Ziemi w ciągu 900 sekund (15 minut) satelitami o swath width równym 800 km.  

Użycie `cross_1_point=False` oznacza, że zamiast używać krzyżowania `1-point-crossover`, będziemy korzystać z `bit-flip`, natomiast podanie `file_name` oznacza, że w każdej iteracji parametry najlepszej znalezionej dotychczas konstelacji będą zapisywane do pliku o nazwie `file_name` (w powyższym przypadku, do pliku `sat_params.txt`).

---

Poszukując bardziej optymalnego rozwiązania niż to policzone powyżej, zbudowaliśmy algorytm genetyczny. W tym celu:

1. Zbudowaliśmy w Pythonie klasę `Planet`, która przechowuje stałe planetarne (jak np. promień Ziemi czy okres rotacyjny) oraz umożliwia dodawanie stacji naziemnych;

2. Zbudowaliśmy również klasę `Orbit`, która przechowuje podstawowe informacje o orbicie, po któej porusza się satelita, takie jak jej parametry, pełną trajektorię (tj. wszystkie możliwe położenia satelity na orbicie) czy okres orbitalny;

3. Z klasy `Orbit` dziedziczy klasa `Satellite`, której obiekty będą reprezentować pojedyńczego satelitę. W ten sposób dla każdego satelity łatwo możemy przechowywać dane jego orbity, jego położenie na orbicie, jak również symulować trajektorię pokonaną na orbicie w danym czasie czy obszar na powierzchni Ziemi, który w tym czasie zdąży pokryć zdjęciami.

4. Dla danej konstelacji satelit, za pomocą klasy `CoverageSimulator` oraz jej metody `calculate_coverage` jesteśmy w stanie wyliczyć, jaką proporcję powierzchni Ziemi dana konstelacja pokryje zdjęciami w danym przedziale czasowym.

Algorytm genetyczny rozważa populację rozmiaru `popSize`. Osobnikiem populacji jest konstelacja satelitów ustalonego rozmiaru `constellation_size` (tj. każda konstelacja składa się z dokładnie `constellation_size` satelitów). Algorytm krzyżuje i mutuje osobniki, zmieniając parametry orbit satelitów, w celu maksymalizacji osiąganego przez konstelację pokrycia Ziemi (funkcja `calculate_coverage`). W szczególności:

1. Początkowo **populacja** składa się z konstelacji złożonych z losowych satelitów (tj. parametry ich orbit są losowane, przy zachowaniu własności heliosynchroniczności). Tworzenie losowych satelitów ułatwia metoda `random_satellite`. Konstelacja jest listą obiektów klasy `Satellit`. Populacja jest listą konstelacji, czyli listą list obiektów klasy `Satellit`.

2. **Ewaluacja konstelacji** polega na policzeniu pokrytej przez nią proporcji Ziemi w danym czasie.

3. **Mutacja konstelacji** jest rozpatrywana osobno dla każdego tworzącego ją satelity i polega na losowej, małej zmianie każdego jego parametru. Satelita mutuje z prawdopodobieństwem `pm`.

4. Zastosowaliśmy dwa rodzaje krzyżowania osobników. Początkowo, korzystaliśmy z **one-point-crossover**. W tym podejściu dwie konstelacje krzyżują się w ten sposób, że przy losowo dobranej wartości `k`, pierwszy potomek dziedziczy pierwszych `k` satelitów pierwszego rodzica, a resztę konstelacji z drugiego rodzica, natomiast drugi potomek na odwrót -- pierwszych `k` satelitów otrzymuje od drugiego rodzica, a pozostałe od pierwszego. Takie podejście wprodzadza jednak relatywnie małą zmienność w populacji. Postanowiliśmy więc przetestować również **bit-flip crossover** -- wersję krzyżowania, która dla każdego satelity z osobna decyduje, czy będzie należał do pierwszego potomka czy do drugiego, z równym prawdopodobieństwem. Dzięki temu konstelacje wymieniają się częścią satelitów (średnio połową, nie muszą to jednak być sąsiednie satelity). Zauważmy, że bit-flip crossover dopuszcza takie samo krzyżowanie jak one-point-crossover (tzn. przy odpowiednim losowaniu możliwe jest, żeby satelity wymieniły się akurat początkowymi `k` satelitami), jednak jest pozbawione ograniczenia w postaci wymiany jedynie sąsiednich satelit -- każda wymiana jest możliwa. W żadnym podejściu nie zmieniamy natomiast parametrów satelitów.

5. Wybór rodziców do krzyżowanie przebiega w ramach tzw. **tournament selection**. Z całej populacji losowanych jest k osobników, a następnie do krzyżowania wybierany jest najlepszy z nich (w sensie funkcji ewaluacji, czyli pokrycia). W ten sposób uzyskujemy jednego rodzica. Procedurę powtarzamy dla drugiego rodzica. Domyślnie `k`=2.

6. Stosujemy standardową **funkcję przetrwania**, tj. po każdej iteracji algorytmu, łączymy aktualną populację z populacją stworzonych w danej iteracji dzieci, oraz wybieramy `popSize` najlepszych osobników, którzy przetrwają, tworząc populację w następnej iteracji.

Powyższe podejście, ze względu na sposób implemntacji (jako klasa) umożliwia w przyszłości łatwe modyfikacje oraz czytelność kodu. W szczególności, można by było uwzględnić w funkcji ewaluacji preferencję dotyczącą pokrycia _ważnych obszarów świata_, takich tak Europa czy Stany Zjednoczone, albo zastosować wyszukiwanie binarne do znalezienia minimalnej liczby satalitów potrzebnych do pokrycia całej Ziemi (albo pożądanego odsetka Ziemi) w ustalonym przedziale czasowym.

---

Nasz algorytm przetestowaliśmy dla różnej liczby satelit w konstelacji, oraz następującej kombinacji parametrów:

`planet` = earth,

`popSize` = 20,

`numParents` = 8,

`pm` = 0.3,

`time_span` = 21600, # 21600 sekund = 6 godzin

`swath_width` = 800,

`numIterations` = 30

Wyniki eksperymentu dla początkowych 16 iteracji przedstawia poniższa tabela:

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;width:100%;height:100%;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
.tg .tg-thickborder{border-left:2px solid black;}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-0pky">No. Satellites</th>
    <th class="tg-c3ow"></th>
    <th class="tg-c3ow">20</th>
    <th class="tg-c3ow tg-thickborder"></th> <!-- And here -->
    <th class="tg-c3ow"></th>
    <th class="tg-c3ow">30</th>
    <th class="tg-c3ow tg-thickborder"></th> <!-- And here -->
    <th class="tg-c3ow"></th>
    <th class="tg-c3ow">40</th>
    <th class="tg-c3ow tg-thickborder"></th> <!-- And here -->
    <th class="tg-c3ow"></th>
    <th class="tg-c3ow">50</th>
    <th class="tg-c3ow tg-thickborder"></th> <!-- And here -->
  </tr>
  <tr>
    <th class="tg-0pky">Iteration</th>
    <th class="tg-c3ow">min</th>
    <th class="tg-c3ow">max</th>
    <th class="tg-c3ow">average</th>
    <th class="tg-c3ow">min</th>
    <th class="tg-c3ow">max</th>
    <th class="tg-c3ow">average</th>
    <th class="tg-c3ow">min</th>
    <th class="tg-c3ow">max</th>
    <th class="tg-c3ow">average</th>
    <th class="tg-c3ow">min</th>
    <th class="tg-c3ow">max</th>
    <th class="tg-c3ow">average</th>

  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0pky">Initial</td>
    <td class="tg-0pky">0.233</td>
    <td class="tg-0pky">0.625</td>
    <td class="tg-0pky">0.349</td>
    <td class="tg-0pky">0.356</td>
    <td class="tg-0pky">0.614</td>
    <td class="tg-0pky">0.484</td>
    <td class="tg-0pky">0.418</td>
    <td class="tg-0pky">0.673</td>
    <td class="tg-0pky">0.554</td>
    <td class="tg-0pky">0.527</td>
    <td class="tg-0pky">0.792</td>
    <td class="tg-0pky">0.641</td>
  </tr>
  <tr>
    <td class="tg-0pky">1</td>
    <td class="tg-0pky">0.299</td>
    <td class="tg-0pky">0.625</td>
    <td class="tg-0pky">0.41</td>
    <td class="tg-0pky">0.428</td>
    <td class="tg-0pky">0.648</td>
    <td class="tg-0pky">0.521</td>
    <td class="tg-0pky">0.523</td>
    <td class="tg-0pky">0.677</td>
    <td class="tg-0pky">0.601</td>
    <td class="tg-0pky">0.591</td>
    <td class="tg-0pky">0.804</td>
    <td class="tg-0pky">0.696</td>
  </tr>
  <tr>
    <td class="tg-0pky">2</td>
    <td class="tg-0pky">0.364</td>
    <td class="tg-0pky">0.625</td>
    <td class="tg-0pky">0.453</td>
    <td class="tg-0pky">0.482</td>
    <td class="tg-0pky">0.654</td>
    <td class="tg-0pky">0.564</td>
    <td class="tg-0pky">0.578</td>
    <td class="tg-0pky">0.759</td>
    <td class="tg-0pky">0.639</td>
    <td class="tg-0pky">0.659</td>
    <td class="tg-0pky">0.841</td>
    <td class="tg-0pky">0.736</td>
  </tr>
  <tr>
    <td class="tg-0pky">3</td>
    <td class="tg-0pky">0.435</td>
    <td class="tg-0pky">0.625</td>
    <td class="tg-0pky">0.493</td>
    <td class="tg-0pky">0.531</td>
    <td class="tg-0pky">0.733</td>
    <td class="tg-0pky">0.606</td>
    <td class="tg-0pky">0.616</td>
    <td class="tg-0pky">0.759</td>
    <td class="tg-0pky">0.657</td>
    <td class="tg-0pky">0.696</td>
    <td class="tg-0pky">0.841</td>
    <td class="tg-0pky">0.777</td>
  </tr>
  <tr>
    <td class="tg-0pky">4</td>
    <td class="tg-0pky">0.473</td>
    <td class="tg-0pky">0.625</td>
    <td class="tg-0pky">0.518</td>
    <td class="tg-0pky">0.569</td>
    <td class="tg-0pky">0.733</td>
    <td class="tg-0pky">0.635</td>
    <td class="tg-0pky">0.633</td>
    <td class="tg-0pky">0.777</td>
    <td class="tg-0pky">0.677</td>
    <td class="tg-0pky">0.764</td>
    <td class="tg-0pky">0.848</td>
    <td class="tg-0pky">0.801</td>
  </tr>
  <tr>
    <td class="tg-0pky">5</td>
    <td class="tg-0pky">0.477</td>
    <td class="tg-0pky">0.625</td>
    <td class="tg-0pky">0.534</td>
    <td class="tg-0pky">0.599</td>
    <td class="tg-0pky">0.772</td>
    <td class="tg-0pky">0.661</td>
    <td class="tg-0pky">0.651</td>
    <td class="tg-0pky">0.777</td>
    <td class="tg-0pky">0.699</td>
    <td class="tg-0pky">0.789</td>
    <td class="tg-0pky">0.907</td>
    <td class="tg-0pky">0.818</td>
  </tr>
  <tr>
    <td class="tg-0pky">6</td>
    <td class="tg-0pky">0.507</td>
    <td class="tg-0pky">0.651</td>
    <td class="tg-0pky">0.562</td>
    <td class="tg-0pky">0.652</td>
    <td class="tg-0pky">0.772</td>
    <td class="tg-0pky">0.694</td>
    <td class="tg-0pky">0.677</td>
    <td class="tg-0pky">0.777</td>
    <td class="tg-0pky">0.727</td>
    <td class="tg-0pky">0.8</td>
    <td class="tg-0pky">0.907</td>
    <td class="tg-0pky">0.829</td>
  </tr>
  <tr>
    <td class="tg-0pky">7</td>
    <td class="tg-0pky">0.529</td>
    <td class="tg-0pky">0.651</td>
    <td class="tg-0pky">0.581</td>
    <td class="tg-0pky">0.665</td>
    <td class="tg-0pky">0.801</td>
    <td class="tg-0pky">0.731</td>
    <td class="tg-0pky">0.719</td>
    <td class="tg-0pky">0.814</td>
    <td class="tg-0pky">0.749</td>
    <td class="tg-0pky">0.805</td>
    <td class="tg-0pky">0.907</td>
    <td class="tg-0pky">0.846</td>
  </tr>
  <tr>
    <td class="tg-0pky">8</td>
    <td class="tg-0pky">0.556</td>
    <td class="tg-0pky">0.747</td>
    <td class="tg-0pky">0.603</td>
    <td class="tg-0pky">0.707</td>
    <td class="tg-0pky">0.802</td>
    <td class="tg-0pky">0.762</td>
    <td class="tg-0pky">0.734</td>
    <td class="tg-0pky">0.814</td>
    <td class="tg-0pky">0.762</td>
    <td class="tg-0pky">0.837</td>
    <td class="tg-0pky">0.907</td>
    <td class="tg-0pky">0.865</td>
  </tr>
  <tr>
    <td class="tg-0pky">9</td>
    <td class="tg-0pky">0.568</td>
    <td class="tg-0pky">0.747</td>
    <td class="tg-0pky">0.622</td>
    <td class="tg-0pky">0.758</td>
    <td class="tg-0pky">0.839</td>
    <td class="tg-0pky">0.783</td>
    <td class="tg-0pky">0.747</td>
    <td class="tg-0pky">0.847</td>
    <td class="tg-0pky">0.774</td>
    <td class="tg-0pky">0.859</td>
    <td class="tg-0pky">0.907</td>
    <td class="tg-0pky">0.877</td>
  </tr>
  <tr>
    <td class="tg-0pky">10</td>
    <td class="tg-0pky">0.588</td>
    <td class="tg-0pky">0.747</td>
    <td class="tg-0pky">0.646</td>
    <td class="tg-0pky">0.771</td>
    <td class="tg-0pky">0.839</td>
    <td class="tg-0pky">0.79</td>
    <td class="tg-0pky">0.758</td>
    <td class="tg-0pky">0.847</td>
    <td class="tg-0pky">0.788</td>
    <td class="tg-0pky">0.865</td>
    <td class="tg-0pky">0.93</td>
    <td class="tg-0pky">0.894</td>
  </tr>
  <tr>
    <td class="tg-0pky">11</td>
    <td class="tg-0pky">0.606</td>
    <td class="tg-0pky">0.747</td>
    <td class="tg-0pky">0.672</td>
    <td class="tg-0pky">0.788</td>
    <td class="tg-0pky">0.852</td>
    <td class="tg-0pky">0.801</td>
    <td class="tg-0pky">0.759</td>
    <td class="tg-0pky">0.881</td>
    <td class="tg-0pky">0.804</td>
    <td class="tg-0pky">0.891</td>
    <td class="tg-0pky">0.93</td>
    <td class="tg-0pky">0.903</td>
  </tr>
  <tr>
    <td class="tg-0pky">12</td>
    <td class="tg-0pky">0.651</td>
    <td class="tg-0pky">0.747</td>
    <td class="tg-0pky">0.69</td>
    <td class="tg-0pky">0.79</td>
    <td class="tg-0pky">0.865</td>
    <td class="tg-0pky">0.812</td>
    <td class="tg-0pky">0.78</td>
    <td class="tg-0pky">0.881</td>
    <td class="tg-0pky">0.82</td>
    <td class="tg-0pky">0.894</td>
    <td class="tg-0pky">0.93</td>
    <td class="tg-0pky">0.911</td>
  </tr>
  <tr>
    <td class="tg-0pky">13</td>
    <td class="tg-0pky">0.686</td>
    <td class="tg-0pky">0.764</td>
    <td class="tg-0pky">0.708</td>
    <td class="tg-0pky">0.8</td>
    <td class="tg-0pky">0.865</td>
    <td class="tg-0pky">0.822</td>
    <td class="tg-0pky">0.799</td>
    <td class="tg-0pky">0.891</td>
    <td class="tg-0pky">0.835</td>
    <td class="tg-0pky">0.904</td>
    <td class="tg-0pky">0.93</td>
    <td class="tg-0pky">0.917</td>
  </tr>
  <tr>
    <td class="tg-0pky">14</td>
    <td class="tg-0pky">0.69</td>
    <td class="tg-0pky">0.764</td>
    <td class="tg-0pky">0.718</td>
    <td class="tg-0pky">0.802</td>
    <td class="tg-0pky">0.865</td>
    <td class="tg-0pky">0.831</td>
    <td class="tg-0pky">0.818</td>
    <td class="tg-0pky">0.891</td>
    <td class="tg-0pky">0.851</td>
    <td class="tg-0pky">0.914</td>
    <td class="tg-0pky">0.93</td>
    <td class="tg-0pky">0.922</td>
  </tr>
  <tr>
    <td class="tg-0pky">15</td>
    <td class="tg-0pky">0.697</td>
    <td class="tg-0pky">0.764</td>
    <td class="tg-0pky">0.724</td>
    <td class="tg-0pky">0.827</td>
    <td class="tg-0pky">0.865</td>
    <td class="tg-0pky">0.847</td>
    <td class="tg-0pky">0.839</td>
    <td class="tg-0pky">0.891</td>
    <td class="tg-0pky">0.864</td>
    <td class="tg-0pky">0.92</td>
    <td class="tg-0pky">0.93</td>
    <td class="tg-0pky">0.926</td>
  </tr>
  <tr>
    <td class="tg-0pky">16</td>
    <td class="tg-0pky">0.721</td>
    <td class="tg-0pky">0.764</td>
    <td class="tg-0pky">0.733</td>
    <td class="tg-0pky">0.833</td>
    <td class="tg-0pky">0.866</td>
    <td class="tg-0pky">0.859</td>
    <td class="tg-0pky">0.85</td>
    <td class="tg-0pky">0.897</td>
    <td class="tg-0pky">0.872</td>
    <td class="tg-0pky">0.921</td>
    <td class="tg-0pky">0.931</td>
    <td class="tg-0pky">0.928</td>
  </tr>
</tbody>
</table>

Nasz algorytm, podczas każdej iteracji wylicza i wyświetla minimalny, maksymalny i średni wynik osiągnięty w populacji. Jak widać w powyższej tabeli, w każdym z czterech przypadków (20, 30, 40 i 50 satelitów w konstelacji), z każdą iteracją wyniki osiągnięte przez populację były coraz lepsze. W szczególności, udało się polepszyć maksymalny wynik pierwszego, losowego satelity z **65.5%** pokrycia Ziemi do **76.5%** w przypadku dwudziestu, z **61.4%** do **86.6%** w przypadku trzydziestu, z **67.3%** do **89.7%** w przypadku czterdziestu, oraz z **79.2%** do **93.1%** w przypadku pięćdziesięciu satelitów. Jednocześnie, wybierając konstelację zupełnie losowo i bez żadnego algorytmu optymalizacyjnego, w przypadku dwudziestu satelitów możemy osiągnąć wyniki tak niskie jak **23.3%** w porównaniu do **76.4%** (najlepszego osiągniętego po szesnastu iteracjach), w przypadku trzydziestu -- **35.6%** w porównaniu do **86.6%**, w przypadku czterdziestu -- **41.8%** w porównaniu do **89.7%**, w przypadku pięćdziesiąciu -- **52.7%** w porównaniu do **93.1%**.

---

W celu porównania algorytmu genetycznego z zaproponowanym wcześniej, prostszym podejściem, przeprowadziliśmy symulację optymalizacji 128 satelit w ciągu 15 minut. Wyniki eksperymentu przestawia poniższa tabela:

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-7zrl{text-align:left;vertical-align:bottom}
.tg .tg-8d8j{text-align:center;vertical-align:bottom}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-7zrl">No. Satellites</th>
    <th class="tg-8d8j" colspan="3">128</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-7zrl">Iteration</td>
    <td class="tg-7zrl">min</td>
    <td class="tg-7zrl">max</td>
    <td class="tg-7zrl">average</td>
  </tr>
  <tr>
    <td class="tg-7zrl">Initial</td>
    <td class="tg-7zrl">0.697</td>
    <td class="tg-7zrl">0.889</td>
    <td class="tg-7zrl">0.789</td>
  </tr>
  <tr>
    <td class="tg-7zrl">1</td>
    <td class="tg-7zrl">0.763</td>
    <td class="tg-7zrl">0.889</td>
    <td class="tg-7zrl">0.816</td>
  </tr>
  <tr>
    <td class="tg-7zrl">2</td>
    <td class="tg-7zrl">0.791</td>
    <td class="tg-7zrl">0.9</td>
    <td class="tg-7zrl">0.838</td>
  </tr>
  <tr>
    <td class="tg-7zrl">3</td>
    <td class="tg-7zrl">0.828</td>
    <td class="tg-7zrl">0.911</td>
    <td class="tg-7zrl">0.861</td>
  </tr>
  <tr>
    <td class="tg-7zrl">4</td>
    <td class="tg-7zrl">0.845</td>
    <td class="tg-7zrl">0.916</td>
    <td class="tg-7zrl">0.881</td>
  </tr>
  <tr>
    <td class="tg-7zrl">5</td>
    <td class="tg-7zrl">0.865</td>
    <td class="tg-7zrl">0.918</td>
    <td class="tg-7zrl">0.897</td>
  </tr>
  <tr>
    <td class="tg-7zrl">6</td>
    <td class="tg-7zrl">0.889</td>
    <td class="tg-7zrl">0.918</td>
    <td class="tg-7zrl">0.905</td>
  </tr>
  <tr>
    <td class="tg-7zrl">7</td>
    <td class="tg-7zrl">0.9</td>
    <td class="tg-7zrl">0.922</td>
    <td class="tg-7zrl">0.91</td>
  </tr>
  <tr>
    <td class="tg-7zrl">8</td>
    <td class="tg-7zrl">0.908</td>
    <td class="tg-7zrl">0.922</td>
    <td class="tg-7zrl">0.913</td>
  </tr>
  <tr>
    <td class="tg-7zrl">9</td>
    <td class="tg-7zrl">0.91</td>
    <td class="tg-7zrl">0.929</td>
    <td class="tg-7zrl">0.914</td>
  </tr>
  <tr>
    <td class="tg-7zrl">10</td>
    <td class="tg-7zrl">0.911</td>
    <td class="tg-7zrl">0.929</td>
    <td class="tg-7zrl">0.917</td>
  </tr>
  <tr>
    <td class="tg-7zrl">11</td>
    <td class="tg-7zrl">0.913</td>
    <td class="tg-7zrl">0.929</td>
    <td class="tg-7zrl">0.918</td>
  </tr>
  <tr>
    <td class="tg-7zrl">12</td>
    <td class="tg-7zrl">0.914</td>
    <td class="tg-7zrl">0.929</td>
    <td class="tg-7zrl">0.92</td>
  </tr>
  <tr>
    <td class="tg-7zrl">13</td>
    <td class="tg-7zrl">0.917</td>
    <td class="tg-7zrl">0.929</td>
    <td class="tg-7zrl">0.922</td>
  </tr>
  <tr>
    <td class="tg-7zrl">14</td>
    <td class="tg-7zrl">0.92</td>
    <td class="tg-7zrl">0.931</td>
    <td class="tg-7zrl">0.924</td>
  </tr>
  <tr>
    <td class="tg-7zrl">15</td>
    <td class="tg-7zrl">0.922</td>
    <td class="tg-7zrl">0.931</td>
    <td class="tg-7zrl">0.927</td>
  </tr>
  <tr>
    <td class="tg-7zrl">16</td>
    <td class="tg-7zrl">0.929</td>
    <td class="tg-7zrl">0.931</td>
    <td class="tg-7zrl">0.93</td>
  </tr>
  <tr>
    <td class="tg-7zrl">17</td>
    <td class="tg-7zrl">0.929</td>
    <td class="tg-7zrl">0.931</td>
    <td class="tg-7zrl">0.93</td>
  </tr>
  <tr>
    <td class="tg-7zrl">18</td>
    <td class="tg-7zrl">0.93</td>
    <td class="tg-7zrl">0.931</td>
    <td class="tg-7zrl">0.931</td>
  </tr>
  <tr>
    <td class="tg-7zrl">19</td>
    <td class="tg-7zrl">0.931</td>
    <td class="tg-7zrl">0.931</td>
    <td class="tg-7zrl">0.931</td>
  </tr>
  <tr>
    <td class="tg-7zrl">20</td>
    <td class="tg-7zrl">0.931</td>
    <td class="tg-7zrl">0.931</td>
    <td class="tg-7zrl">0.931</td>
  </tr>
</tbody>
</table>

Mimo że algorytmowi udało się polepszyć początkowy wynik (88.9% $\rightarrow$ 93.1%) to zmiana nie jest duża. Zapewne wynika to z tego, że gdy dana konstelacja osiąga relatywnie wysoki poziom pokrycia Ziemi (np. 88.9%, jak w tym przypadku) to niewidziany przez nią obszar jest mały, więc aby go pokryć, potrzebne jest włączenie do konstelacji szczególnego, konkretnego satelity, który "specjalizowałby się" w obserwacji tego konkretnego, niewidzianego obszaru. W algorytmie genetycznym wyposażenie konstelacji w takiego satelitę może nastąpić jedynie w wyniku krzyżowania lub w wyniku mutacji. Krzyżowanie wymaga, by rozpatrywana przez nas konstelacja została wylosowana jako rodzic wraz z inną konstelacją, która akurat takiego satelitę posiada, a następnie  wymieniła z nią ten konkretny gen (w przypadku `bit-flip` -- z prawdopodobieństwem 0.5). Szansa na tak korzystne krzyżowanie jest zatem niewielka.

W przypadku mutacji sytuacja jest podobna -- wynik konstelacji polepszy się, jeżeli w wyniku mutacji genu uda się pokryć niewidziany dotychczas obszar, co przy obecnie stosowanej mutacji jest trudne, bo wymaga, by satelita bardzo bliski pożądanemu już znajdował się w konstelacji (mutacja dokonuje bardzo niewielkich zmian parametrów orbit satelitów).

Dodatkowo warto wspomnieć, że obecna implementacja algorytmu rozpatruje każdego satelitę niezależnie (każdy satelita ma własną orbitę). W poprzednim podejściu natomiast zakładaliśmy, że na danej orbicie umieszczamy kilka satelit w równych odstępach. Tego rodzaju podejście ma dwie istotne wady. Po pierwsze, możliwe jest że algorytm umieści kilka satelitów na bardzo podobnych, jednak różnych orbitach, natomiast w praktyce prostsze może być umieszczenie ich na jednej (uśrednionej) orbicie, być może również wyrównując odstępy między satelitami. Po drugie, w tym podejściu algorytm sam musi się nauczyć, że w sytuacji podobnych parametrów orbitalnych satelitów, pożądane jest ustawienie ich we w miarę równych odstępach, co dla algorytmu może być trudne i wymagać wielu iteracji.   

---

Zaletą wybranego algorytmu jest jego elastyczność -- struktura algorytmów genetycznych wymusza, że z każdą iteracją otrzymane wyniki są nie gorsze od poprzednich (zwykle lepsze, zwłaszcza w początkowych iteracjach), a jednocześnie mutowanie genów umożliwia eksplorację przestrzeni rozwiązań (tj. rozważanie satelitów, których w początkowej populacji nie było). Połączenie tych cech daje nadzieję na znalezienie optymalnego rozwiązania przy odpowiednio dużej liczbie iteracji. Z drugiej strony, warte rozważenia byłoby włączenie do algorytmu innych, bardziej "drastycznych" rodzajów mutacji i krzyżowania, na przykład krzyżowania uśredniającego parametry satelit rodziców, albo mutacji zmieniającej satelitę na zupełnie innego, wygenerowanego losowo i nie powiązanego z poprzednim. Takie rozwiązanie pogłębiłoby możliwości eksploracyjne i dawałoby szansę na wyjście z lokalnego optimum po zbiegnięciu algorytmu, co przy zwiększeniu liczby iteracji umożliwiłoby dalsze polepszanie osiągniętych wyników.




Z drugiej strony, największą wadą zaproponowanego rozwiązania jest czas potrzebny na przeprowadzenie symulacji. Głównym problemem jest ewaluacja konstelacji z populacji. Funkcja ewaluacji ma za zadanie ocenić łączną powierzchnię "widzianą" przez całą konstelację. Aby uniknąć kilkukrotnego liczenia powierzchni fotografowanych obszarów w sytuacji, w której obszary widziane przez kilka satelitów nakładają się na siebie, powierzchnie widzianych obszarów nie są sumowane 'pojedynczo'. Zamiast tego, obszary, które mogą zostać sfotografowane przez satelitę są zaimplementowane jako obiekty klasy `Polygon` (z biblioteki `shapely`), tak aby podczas ewaluacji wszystkie obszary zostały połączone w jeden obiekt (również `Polygon`), a dopiero na końcu liczona jest powierzchnia połączonego obszaru. Operacja łączenia wszystkich tych wielokątów jest jednak zadziwiająco czasochłonna.


## Wizualizacja algorytmu genetycznego

Za pomocą biblioteki poliastro stworzyliśmy wizualizacje miejsc, nad którymi przelytywały satelity o wyliczonych orbitach przez algorytm genetyczny.

In [1]:
dane_orbit = 'sat_params_30.txt' #aby zobaczyć wizualizację dla 24/30 satelitów należy zmienić zmienną na 'sat_params24/30.txt'
df = pd.read_csv(dane_orbit, sep = ' ', names = ['a', 'ecc', 'inc', 'raan', 'argp', 'nu', 'where'])
planet_radius = 6371
orbits = []
new_satelites = []
times = []
n = df.shape[0] #liczba satelitów
our_time = 3 #co ile godzin chcemy aktualizować obraz z danego obszaru
for i in range(n):
    a = df['a'][i] / 1000.0 * u.km #wielka półoś
    ecc = df['ecc'][i] * u.one #kkscentryczność
    inc = df['inc'][i] * 57.2957795 * u.deg #inklinacja
    raan = df['raan'][i] * 57.2957795 * u.deg #raan
    nu = df['nu'][i] * u.deg #prawdziwa anomalia
    argp = df['argp'][i] * 57.2957795 * u.deg #argument perycentrum
    where = df['where'][i] #miejsce startu satelity
                           #(po ilu sekundach znalazłby się w tym miejscu, gdyby startował z równika)
    orbits.append(Orbit.from_classical(Earth, a, ecc, inc, raan, argp, nu, iss.epoch))
    new_satelites.append(EarthSatellite(orbits[i], None))
    times.append(time_range(
        iss.epoch + (where / 3600.0) * u.h, periods=150, end=iss.epoch + (our_time + where / 3600.0) * u.h )) #chodzi nam o epokę ISS i chcemy wobec niej ustalać czas
# Generate an instance of the plotter, add title and show latlon grid
gp = GroundtrackPlotter()
gp.update_layout(title="Optymalne orbity 30 satelitów")
from colour import Color
red = Color("red")
colors = list(red.range_to(Color("green"),n))

# Plot previously defined EarthSatellite object
for i in range(n):
    gp.plot(
        new_satelites[i],
        times[i],
        label="",
        color=str(colors[i]),
        line_style = {"width": 1 , "color": str(colors[i])},
        marker={
            "size": 0,
            "symbol": "triangle-right",
            "line": {"width": 0, "color": "black"},
        },
    )
import plotly.graph_objects as go

WWA = [52.2297, 21.0122] * u.deg #można również dodawać położenie miast lub obiektów znając ich współrzędne geograficzne
gp.add_trace(
    go.Scattergeo(
        lat=WWA[0],
        lon=WWA[-1],
        name="WWA",
        marker={"color": "blue"},
        )
)

gp.fig.show()

NameError: name 'pd' is not defined

Można również zobaczyć otrzymane tory lotu w trójwymiarowej wizualizacji.

In [ ]:
gp.update_geos(projection_type="orthographic")
gp.fig.show()

##  Tradeoff revisit.
Za pomocą algorytmu genetycznego otrzymaliśmy kilka zadowalających wyników.
Dla 30 satelitów odwiedzających każde miejsce co 6 godziny byliśmy w stanie pokryć 86.6% powierzchni całej Ziemii. Natomiast zmieniając dane na 24 satelity i odwiedzanie co 3 godziny uzyskaliśmy jedynie 75% widocznej powierzchni planety.

* [1] https://earth.esa.int/eogateway/missions/worldview-2
* [2] https://earth.esa.int/eogateway/missions/pleiades
* [3] https://pl.wikipedia.org/wiki/Anomalia_prawdziwa
* [4] https://en.wikipedia.org/wiki/Sun-synchronous_orbit
* [5] https://en.wikipedia.org/wiki/Orbital_inclination
* [6] https://en.wikipedia.org/wiki/Precession
* [7] https://en.wikipedia.org/wiki/Earth_ellipsoid
* [8] https://en.wikipedia.org/wiki/Haversine_formula